In [12]:
import pandas as pd
import re
from nltk.corpus import stopwords
import nltk.data

In [13]:
# Read data from files 
train = pd.read_csv( "train@urlpandaslower.csv", header=0, 
 delimiter=",", quoting=0 )
test = pd.read_csv( "test@urlpandaslower.csv", header=0, delimiter=",", quoting=0 )



In [14]:
# Verify the number of reviews that were read (100,000 in total)
print "Read %d labeled train messages, %d labeled test messages\n " \
    % (train["Message"].size,  test["Message"].size )

Read 1600000 labeled train messages, 498 labeled test messages
 


In [15]:
def message_to_wordlist( message, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    #review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    message_text = re.sub("[^a-zA-Z]"," ", message)
    #
    # 3. Convert words to lower case and split them
    words = message_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [16]:
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def message_to_sentences( message, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(message.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( message_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [18]:
sentences = []  # Initialize an empty list of sentences

print "Parsing sentences from training set"
for message in train["Message"]:
    sentences += message_to_sentences(message, tokenizer)

#print "Parsing sentences from unlabeled set"
#for message in unlabeled_train["Message"]:
#    sentences += message_to_sentences(message, tokenizer)


Parsing sentences from training set


In [19]:
print len(sentences)


2701164


In [ ]:
print sentences[0]

In [21]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 400    # Word vector dimensionality                      
min_word_count = 50   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print "Training model..."
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)


Training model...


In [22]:
model.doesnt_match("man woman child kitchen".split())

'kitchen'

In [23]:
model.doesnt_match("france england germany berlin".split())
'berlin'

'berlin'

In [24]:
model.doesnt_match("paris berlin london austria".split())
'paris'

'paris'

In [25]:
model.most_similar("awful")

[('unpleasant', 0.4942152500152588),
 ('horrible', 0.4806709885597229),
 ('understatement', 0.47916990518569946),
 ('dreadful', 0.4743342399597168),
 ('horrid', 0.470512717962265),
 ('terrible', 0.46351492404937744),
 ('shocking', 0.45535731315612793),
 ('accomplishment', 0.45230770111083984),
 ('intriguing', 0.44709300994873047),
 ('positively', 0.44331568479537964)]

In [26]:
# Load the model that we created in Part 2
from gensim.models import Word2Vec
model = Word2Vec.load("300features_40minwords_10context")

In [27]:
type(model.syn0)

numpy.ndarray

In [28]:
model.syn0.shape

(12648, 400)

In [29]:
from sklearn.cluster import KMeans
import time

start = time.time() # Start time

# Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an
# average of 5 words per cluster
word_vectors = model.syn0
num_clusters = word_vectors.shape[0] / 5

# Initalize a k-means object and use it to extract centroids
kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vectors )

# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print "Time taken for K Means clustering: ", elapsed, "seconds."

In [ ]:
# Create a Word / Index dictionary, mapping each vocabulary word to
# a cluster number                                                                                            
word_centroid_map = dict(zip( model.index2word, idx ))

In [ ]:
# For the first 10 clusters
for cluster in xrange(0,10):
    #
    # Print the cluster number  
    print "\nCluster %d" % cluster
    #
    # Find all of the words for that cluster number, and print them out
    words = []
    for i in xrange(0,len(word_centroid_map.values())):
        if( word_centroid_map.values()[i] == cluster ):
            words.append(word_centroid_map.keys()[i])
    print words

In [ ]:
Cluster 0

In [ ]:
def create_bag_of_centroids( wordlist, word_centroid_map ):
    #
    # The number of clusters is equal to the highest cluster index
    # in the word / centroid map
    num_centroids = max( word_centroid_map.values() ) + 1
    #
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    #
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count 
    # by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    #
    # Return the "bag of centroids"
    return bag_of_centroids

In [ ]:
# Pre-allocate an array for the training set bags of centroids (for speed)
train_centroids = np.zeros( (train["Message"].size, num_clusters), \
    dtype="float32" )

# Transform the training set reviews into bags of centroids
counter = 0
for message in clean_train_messages:
    train_centroids[counter] = create_bag_of_centroids( message, \
        word_centroid_map )
    counter += 1

# Repeat for test reviews 
test_centroids = np.zeros(( test["Message"].size, num_clusters), \
    dtype="float32" )

counter = 0
for message in clean_test_messages:
    test_centroids[counter] = create_bag_of_centroids( message, \
        word_centroid_map )
    counter += 1

In [ ]:
# Fit a random forest and extract predictions 
forest = RandomForestClassifier(n_estimators = 100)

# Fitting the forest may take a few minutes
print "Fitting a random forest to labeled training data..."
forest = forest.fit(train_centroids,train["Sentiment"])
result = forest.predict(test_centroids)

# Write the test results 
output = pd.DataFrame(data={"Message":test["Message"], "Sentiment":result})
output.to_csv( "BagOfCentroids.csv", index=False, quoting=0 )